# HellaSwag

In [2]:
from unittest.mock import patch
from datasets import load_dataset

with patch("datasets.builder.is_remote_filesystem", return_value=False):
    dataset = load_dataset("Rowan/hellaswag")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/39905 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10042 [00:00<?, ? examples/s]

In [11]:
from dataclasses import dataclass

NUM_ANSWERS = 4
CHOICES = ["A", "B", "C", "D"]


@dataclass
class Sample:
    """A sample multiple choice question."""

    question: str
    answers: list[str]
    label: int

    @property
    def choices(self) -> list[str]:
        """Return the multiple choice answers."""
        return [ f"{choice}) {option}" for choice, option in zip(CHOICES, self.answers) ]

    @property
    def label_text(self) -> str:
        """Returns the expected correct answer."""
        return self.answers[self.label]

    @property
    def expected(self) -> str:
        """Returns the expected correct answer."""
        return f"{CHOICES[self.label]}) {self.label_text}"


def make_sample(record: dict[str, str]) -> Sample:
    return Sample(record["ctx"], record["endings"], label=int(record["label"]))

ds = dataset['train']
it = iter(ds)
make_sample(next(it)), make_sample(next(it)), make_sample(next(it))

(Sample(question='Then, the man writes over the snow covering the window of a car, and a woman wearing winter clothes smiles. then', answers=[', the man adds wax to the windshield and cuts it.', ', a person board a ski lift, while two men supporting the head of the person wearing winter clothes snow as the we girls sled.', ', the man puts on a christmas coat, knitted with netting.', ', the man continues removing the snow on his car.'], label=3),
 Sample(question='A female chef in white uniform shows a stack of baking pans in a large kitchen presenting them. the pans', answers=['contain egg yolks and baking soda.', 'are then sprinkled with brown sugar.', 'are placed in a strainer on the counter.', 'are filled with pastries and loaded into the oven.'], label=3),
 Sample(question='A female chef in white uniform shows a stack of baking pans in a large kitchen presenting them. The pans are filled with pastries and loaded into the oven. a knife', answers=['is seen moving on a board and cutti

# Open AI Client


In [7]:
from openai import OpenAI

BASE_URL = "http://llama-cpp-server-cuda.llama:8000/v1"
API_KEY = "sk-xxx"
MODEL_ID = "/data/models/mistral-7b-inst.gguf"

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {
            "content": "You are a helpful assistant.",
            "role": "system"
        },
        {
            "content": "What is the capital of France?",
            "role": "user"
        }
    ],
)
for choice in response.choices:
    print(choice.message.content)

 The capital city of France is Paris.


In [8]:
INSTRUCTIONS = "Choose the most plausible continuation for the story."
HEADER = "Please answer with the letter of the correct answer."


def multiple_choice_prompt(sample: Sample) -> str:
    """Make a prompt from a sample."""
    return "\n".join([
        HEADER,
        "",
        sample.question,
        "",
        *sample.choices
    ])


def answer(client: OpenAI, sample: Sample) -> str:
    prompt = multiple_choice_prompt(sample)
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {
                "content": INSTRUCTIONS,
                "role": "system"
            },
            {
                "content": prompt,
                "role": "user"
            }
        ],
    )
    choice = response.choices[0]
    return choice.message.content.strip()


sample = make_sample(next(it))
response = answer(client, sample)
sample, response

(Sample(question='A tray of potatoes is loaded into the oven and removed. A large tray of cake is flipped over and placed on counter. a large tray of meat', answers=['is placed onto a baked potato.', ', ls, and pickles are placed in the oven.', 'is poured into a midden.', 'is prepared then it is removed from the oven by a helper when done.'], label='3'),
 'D) is prepared then it is removed from the oven by a helper when done.')

# Evaluate

In [14]:
from tqdm import tqdm

correct = 0
total = 0
it = iter(ds)

In [15]:
pbar = tqdm(enumerate(it))
for i, record in pbar:
    sample = make_sample(next(it))
    response = answer(client, sample)
    total += 1
    # Sometimes the model may answer something like 'The best answer is A) <answer>'
    # so we'll give it credit.
    if response == sample.expected or sample.label_text in response:
        correct += 1
    else:
        print(response)
        print(sample.expected)

    accuracy = (correct / total) * 100
    pbar.set_description(f"{accuracy:2.2f}% - {correct} of {total}")

66.67% - 2 of 3: : 3it [00:04,  1.53s/it] 

B) Then walks over to the sink and smiles while shaking his wet hair.
D) scrubs in the shampoo and then washes it off.


50.00% - 2 of 4: : 4it [00:06,  1.68s/it]

B) lets his hair down, giving it a final blow dry before putting it next to his face.
C) uses an electric clipper to groom the sideburns and the temples.


57.14% - 4 of 7: : 7it [00:12,  1.85s/it]

...

B) then pours the salad into a bigger bowl, and adds dressing.
A) takes the towel and puts it over the lettuce.


43.99% - 216 of 491: : 491it [16:08,  2.15s/it]

A) The person puts the cheese and jam on the bread, then he eats the sandwich and drinks soda from the bottle.
A) , the person puts the cheese and jam on the bread, then he eats the sandwich and drink soda from bottle.


44.22% - 218 of 493: : 493it [16:14,  1.98s/it]


KeyboardInterrupt: 